# All about $3d^1$, $4d^1$, $5d^1$, $3d^9$, $4d^9$, $5d^9$ transition-metal ions in crystals

In [1]:
%load_ext autoreload
# provide cell timings
%load_ext autotime
# %autoreload 2
import sympy as sp
import numpy as np
from scipy.special import sph_harm as Ynm_num
# from sympy import poly, lambdify
from IPython.display import display, Math
# %config InlineBackend.figure_format='retina'
%config Completer.use_jedi = False
from qdef import *
from itertools import product
from sympy import I
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import HTML, display, Math, Latex
from matplotlib import pyplot as plt
%matplotlib widget
from pyperclip import copy as pypercopy

Reloading /Users/juan/Google Drive/Zia Lab/Codebase/qdef/data/CPGs.pkl ...


In [2]:
HartreeFockData.radial_average('Fe',1,2)

0.50061

In [3]:
HartreeFockData.atom_size('Al',6)*UnitCon.con_factor('Å','pm')

25.0

In [8]:
UnitCon.con_factor('THz','Hz')

1000000000000.0

In [22]:
x = 1

In [29]:
class number():
    def __init__(self,value):
        self.value = value
    def convert(self,unit=2):
        return self.value*unit
class Con():
    '''
    Basis constants, all given in standard SI units.
    c, π, ℏ, h, ε0, μ0
    '''
    c = number(299792458)
    π = 3.14159265358
    ℏ = 6.62607015e-34
    h = ℏ*2*π
    ε0 = 8.8541878128e-12
    μ0 = 1.25663706212e-6

In [33]:
Con.c

## A single d-electron in a CF of arbitrary symmetry

```
||| d^1 analysis pseudo-code |||

given a specific point group, construct a model
for the effect that a crystal field with this symmetry
would have on the l=2 levels of a hydrogenic atom
give the results in terms of the smallest set of
parameters that the CF potential should have.

Step 1: construct the symbolic expression for the CF
limiting the expansion to the terms that would have non-zero
matrix elements in the l=2 subspace.

Step 2: find the matrix representation of this potential
in the l=2 subspace

Step 3: Find eigenvalues and eigenvectors.

Step 4: Play around with the eigenvectors to form more convenient linear combos.

HALT

```

### Parsing data from Morrison's book & engineering the CrystalField class

#### Parsing some and making crystal fields

In [304]:
# loader for the data in Morrison's book
# this sheet is an export of the data on 
# a Google Spreadsheet "Morrison Data"
morrison_sheets = pd.read_excel('/Users/juan/Downloads/Morrison.xlsx',None)

In [305]:
group_label_to_index = {gl:(1+CPGs.all_group_labels.index(gl)) for gl in CPGs.all_group_labels}
index_to_group_label = {(1+CPGs.all_group_labels.index(gl)):gl for gl in CPGs.all_group_labels}

In [306]:
morrison = {}
# parse data for the free-ion model
frows = ['nd^N','Ion','charge','F^2','F^4','ζ','α','Reference']
morrison['Free-ion params'] = pd.concat([morrison_sheets['1.'+x][frows] for x in 'A B C'.split(' ')])
morrison['Free-ion params']['charge'] = morrison['Free-ion params']['charge'].astype(int)
# parse data for number of Bnm and Anm as a function of group number
# How many Bnms or Anms for the d^N configuration
# this will just be a dictionary
BnmAnms = morrison_sheets['1.D'][['Group No.','n=1','n=2','n=3','n=4','n=5']]
BnmAnms['Group Label'] = (BnmAnms['Group No.'].apply(lambda x: index_to_group_label[int(x)]))
for i in [1,2,3,4,5]:
    BnmAnms['n=%d' % i] = BnmAnms['n=%d' % i].astype(int)
BnmAnms['Group No.'] = BnmAnms['Group No.'].astype(int)
morrison['BnmAnms'] = BnmAnms
# parse data for Hartree-Fock
hartree = morrison_sheets['2']
metadat = list(morrison_sheets['2'])[0]
hartree = hartree[list(hartree)[1::]]
hartree.attrs['metadata'] = metadat
hartree['N'] = hartree['nd^N'].apply(lambda x: int(x.split('d')[-1]))
hartree['n'] = hartree['nd^N'].apply(lambda x: int(x.split('d')[0]))
hartree['charge'] = hartree['charge'].astype(int)
hartree['Z'] = hartree['Z'].astype(int)
hartree.drop(columns=['nd^N'], inplace=True)
morrison['Hartree-Fock'] = hartree

/Users/juan/opt/anaconda3/envs/qdef/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/juan/opt/anaconda3/envs/qdef/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/Users/juan/opt/anaconda3/envs/qdef/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

In [5]:
# allBees = ['B%d%d.re' % (n,m) for (n,m) in product([1,2,3,4],[0,1,2,3,4])] + ['B%d%d.im' % (n,m) for (n,m) in product([1,2,3,4],[0,1,2,3,4])]

In [307]:
for x in 'A B C D'.split(' '):
    print(list(morrison_sheets['1.'+x]))

['Free-ion data: F^2, F^4, ζ, and α for nd^N ions (cm^-1) (Α) 3δ^ν', 'nd^N', 'Ion', 'charge', 'F^2', 'F^4', 'ζ', 'α', 'Reference']
['Free-ion data: F^2, F^4, ζ, and α for nd^N ions (cm^-1) (B) 4d^N', 'nd^N', 'Ion', 'charge', 'F^2', 'F^4', 'ζ', 'α', 'Reference']
['Free-ion data: F^2, F^4, ζ, and α for nd^N ions (cm^-1) (C) 5d^N', 'nd^N', 'Ion', 'charge', 'F^2', 'F^4', 'ζ', 'α', 'Reference']
['Table 1D. Number of Bnm or Anm for the d^N configuration for 30 of 32 point groups', 'Group No.', 'n=1', 'n=2', 'n=3', 'n=4', 'n=5', 'Host', 'Site']


In [7]:
allBees = []
for n in [1,2,3,4,5,6]:
    for m in range(0,n+1):
        allBees.append('B%d%d.re' % (n,m))
        allBees.append('B%d%d.im' % (n,m))
t8p1v1 = ('\n'.join(morrison_sheets['AngMom8.1and8.2']['8p1'][1:]))
header = morrison_sheets['AngMom8.1and8.2']['8p1'][0]
rows = []
for idx, line in zip(range(3,16),t8p1v1.split('\n')):
    rows.append({what:(x=='x') for what,x in zip(header.split(),line)})
full_params = {}
for group_num, row in zip(range(3,16),rows):
    extension = {bee:False for bee in allBees if bee not in row.keys()}
    row.update(extension)
    full_params[group_num] = dict(row)
t8p2v1 = ('\n'.join(morrison_sheets['AngMom8.1and8.2']['8p2'][1:-1]))
header = morrison_sheets['AngMom8.1and8.2']['8p2'][0]
rows = []
for line in t8p2v1.split('\n'):
    rows.append({what:(x=='x') for what,x in zip(header.split(),line)})
for group_num, row in zip(range(16,28),rows):
    extension = {bee:False for bee in allBees if bee not in row.keys()}
    row.update(extension)
    full_params[group_num] = dict(row)
rows = [
    {'B32.re': True, 'B40.re': True, 'B44.re': True,'B60.re': True, 'B64.re': True},
    {'B40.re': True, 'B44.re': True,'B60.re': True, 'B64.re': True},
    {'B32.re': True, 'B40.re': True, 'B44.re': True,'B60.re': True, 'B64.re': True},
    {'B40.re': True, 'B44.re': True,'B60.re': True, 'B64.re': True},
    {'B32.re': True, 'B40.re': True, 'B44.re': True,'B60.re': True, 'B64.re': True}
]
for group_num, row in zip(range(28,33),rows):
    extension = {bee:False for bee in allBees if bee not in row.keys()}
    row.update(extension)
    full_params[group_num] = dict(row)
def fixer(s):
    see, kind = s.split('.')
    sign = ''
    if '-' in see:
        sign = '-'
        see = see.replace('-','')
    better = '%s_{%s,%s}^%s' % (see[0],see[1:][0],sign+see[1:][1],kind)
    better = better.replace('im','i').replace('re','r')
    return better
full_params = {group_num : {sp.Symbol(fixer(k)): v for k,v in full_params[group_num].items()} for group_num in full_params}
morrison['Bkq grid from tables 8.1-8.3 in_Morrison 1988'] = full_params

In [8]:
cf_components_keys = [k for k in morrison_sheets if 'crystal-field components' in list(morrison_sheets[k])[0].lower()]

In [9]:
crystallographic_sheets = {k.split('.')[0]: list(morrison_sheets[k])[0].split(' on')[-1] for k in morrison_sheets if 'Crystallographic Data' in list(morrison_sheets[k])[0]}

In [10]:
crystallographic_sheets

{'3': ' Y3Al5O12', '4': ' Two Forms of K2NaAlF6'}

In [11]:
for cf_key in cf_components_keys:
    print(cf_key)
    component_data = morrison_sheets[cf_key]
    pivot = list(component_data)[0]
    for_what = (pivot.split('for ')[-1])
    print(for_what)
    subs = for_what.split(' ')[0]
    print(subs)

3.2.1
Al2 (S4) site
Al2
3.3.1
Al1 (C3i) site (rotated so that z-axis is parallel to (111) crystallographic axis)
Al1
20.2.1
Zn (T_d) site of cubic ZnS
Zn
20.2.2
Zn (C3v) site of hexagonal ZnS
Zn
20.2.3
Zn (C3v) site of hexagonal ZnS (Kisi and Elcombe, 1989)
Zn


In [12]:
special_reps = {}
# in all cases 
generic_reps = {sp.Symbol(fixer('B%d%d.re'%(k,q))): (-sp.S(1))**q*sp.Symbol(fixer('B%d%d.re'%(k,-q))) for k in [1,2,3,4,5,6] for q in range(-k,0) }
generic_reps.update({sp.Symbol(fixer('B%d%d.im'%(k,q))): -(-sp.S(1))**q*sp.Symbol(fixer('B%d%d.im'%(k,-q))) for k in [1,2,3,4,5,6] for q in range(-k,0) })
# in some groups some params are related between one another
special_reps = {i:generic_reps for i in range(1,28)}
treps = {sp.Symbol(fixer('B44.re')): sp.sqrt(sp.S(5)/sp.S(14))*sp.Symbol(fixer('B40.re')),
        sp.Symbol(fixer('B64.re')): -sp.sqrt(sp.S(7)/sp.S(2))*sp.Symbol(fixer('B60.re'))}
treps.update(special_reps)
special_reps[28] = treps
special_reps[29] = treps
special_reps[30] = treps
special_reps[31] = treps
special_reps[32] = treps
cnm_reps = {sp.Symbol('C_{%d,%d}'%(k,q)): (sp.S(-1))**q*sp.conjugate(sp.Symbol('C_{%d,%d}'%(k,-q))) for k in [1,2,3,4,5,6] for q in range(-k,0) }
morrison['special_reps'] = special_reps

In [13]:
crystal_fields = {}
for i in range(1,33):
    crystal_fields[i] = compute_crystal_field(i)

Too little symmetry, returning empty list.
Too little symmetry, returning empty list.


In [14]:
# def compute_crystal_field_debug(group_num):
#     '''
#     This function returns a list with the possible forms that the
#     crystal field has for the given group. This list has only one
#     element up till group 27, after that the list has two  possi-
#     bilities that express the possible sign relationships between
#     the B4q and the B6q coefficients.
#     For groups 1-3 an empty list is returned.
#     The crystal field is a qet which has as keys tuples (k,q) and
#     as values sympy symbols for the corresponding coefficients.
#     '''
#     full_params = morrison['Bkq grid from tables 8.1-8.3 in_Morrison 1988']
#     if group_num < 3:
#         print("Too little symmetry, returning empty list.")
#         return []
#     if group_num <=27:
#         cf = []
#         for k,v in full_params[group_num].items():
#             the_subs = {k:0 for k,v in full_params[group_num].items() if not v}
#             print(the_subs)
#             special_reps = morrison['special_reps'][group_num]
#             acf = generic_cf.subs(special_reps).subs(the_subs)
#             cf.append(acf)
# #         cf = [generic_cf.subs(morrison['special_reps'][group_num]).subs({k:0 for k,v in full_params[group_num].items() if not v})]
#     elif group_num == 28:
#         cf_p = Qet({
#                  (3,2): sp.Symbol('B_{3,2}^r'),
#                  (3,-2): sp.Symbol('B_{3,2}^r'),
#                  (4,0): sp.Symbol('B_{4,0}^r'),
#                  (4,4): sp.sqrt(sp.S(5)/sp.S(14))*sp.Symbol('B_{4,0}^r'),
#                  (4,-4): sp.sqrt(sp.S(5)/sp.S(14))*sp.Symbol('B_{4,0}^r'),
#                  (6,0): sp.Symbol('B_{6,0}^r'),
#                  (6,4): -sp.sqrt(sp.S(7)/sp.S(2))*sp.Symbol('B_{6,0}^r'),
#                  (6,-4): -sp.sqrt(sp.S(7)/sp.S(2))*sp.Symbol('B_{6,0}^r'),
#                  })
#         cf_m = Qet({
#                  (3,2): sp.Symbol('B_{3,2}^r'),
#                  (3,-2): sp.Symbol('B_{3,2}^r'),
#                  (4,0): sp.Symbol('B_{4,0}^r'),
#                  (4,4): -sp.sqrt(sp.S(5)/sp.S(14))*sp.Symbol('B_{4,0}^r'),
#                  (4,-4): -sp.sqrt(sp.S(5)/sp.S(14))*sp.Symbol('B_{4,0}^r'),
#                  (6,0): sp.Symbol('B_{6,0}^r'),
#                  (6,4): sp.sqrt(sp.S(7)/sp.S(2))*sp.Symbol('B_{6,0}^r'),
#                  (6,-4): sp.sqrt(sp.S(7)/sp.S(2))*sp.Symbol('B_{6,0}^r'),
#                  })
#         cf = [cf_p, cf_m]
#     elif group_num == 29:
#         cf_p = Qet({
#                  (4,0): sp.Symbol('B_{4,0}^r'),
#                  (4,4): sp.sqrt(sp.S(5)/sp.S(14))*sp.Symbol('B_{4,0}^r'),
#                  (4,-4): sp.sqrt(sp.S(5)/sp.S(14))*sp.Symbol('B_{4,0}^r'),
#                  (6,0): sp.Symbol('B_{6,0}^r'),
#                  (6,4): -sp.sqrt(sp.S(7)/sp.S(2))*sp.Symbol('B_{6,0}^r'),
#                  (6,-4): -sp.sqrt(sp.S(7)/sp.S(2))*sp.Symbol('B_{6,0}^r'),
#                  })
#         cf_m = Qet({
#                  (4,0): sp.Symbol('B_{4,0}^r'),
#                  (4,4): -sp.sqrt(sp.S(5)/sp.S(14))*sp.Symbol('B_{4,0}^r'),
#                  (4,-4): -sp.sqrt(sp.S(5)/sp.S(14))*sp.Symbol('B_{4,0}^r'),
#                  (6,0): sp.Symbol('B_{6,0}^r'),
#                  (6,4): sp.sqrt(sp.S(7)/sp.S(2))*sp.Symbol('B_{6,0}^r'),
#                  (6,-4): sp.sqrt(sp.S(7)/sp.S(2))*sp.Symbol('B_{6,0}^r'),
#                  })
#         cf = [cf_p, cf_m]
#     elif group_num == 30:
#         cf = compute_crystal_field(29)
#     elif group_num == 31:
#         cf = compute_crystal_field(28)
#     elif group_num == 32:
#         cf = compute_crystal_field(30)
#     return cf

In [44]:
full_params = morrison['Bkq grid from tables 8.1-8.3 in_Morrison 1988']

In [15]:
# this bit of code is necessary to make a raw (just dictionaries) of the
# crystal fields
# its a bit awkward
crystal_fields_raw = {}
for k in crystal_fields:
    if k in [1,2]:
        crystal_fields_raw[k] = []
    else:
        crystal_fields_raw[k] = [q.dict for q in crystal_fields[k]]
morrison['crystal_fields_raw'] = crystal_fields_raw       

In [17]:
pickle.dump(morrison,open('/Users/juan/Google Drive/Zia Lab/Codebase/qdef/data/morrison.pkl','wb'))

In [87]:
def irtransform(eigenbasis, l, group_num):
    # given a basis eigenbasis for a subspace for a fixed value of l
    # and the index group_num for a point group
    # return a dictionary whose keys are the 
    # irreducible representation of the group
    # and whose values are dictionaries whose
    # keys are labels for group basis vectors
    # and whose values are linear combinations
    # of the elements in lbasis
    # the lbasis is as the set of coefficients
    # for the ordered basis provided by mbasis(l)
    group = CPGs.groups[group_num]

In [101]:
def fromvectoqet(orderedbasis,coeffvec):
    '''
    Given a vector of coefficients and 
    its corresponding ordered basis,
    return what would be the corresponding
    Qet.
    '''
    return Qet({ovec: coeff for ovec, coeff in zip(orderedbasis, coeffvec)})

In [91]:
eigenbasis = eigensys[1][1][2]

In [100]:
fromvectoqet(mbasis(2,True),eigenbasis[1])

Qet({(2, -2): -1, (2, 2): 1})

In [75]:
def eigen_latex(eigensys, l, normalized=False):
    eigen_chunks = []
    mbase = mbasis(l)
    for eigen_chunk in eigensys:
        eigenval_latex = sp.latex(Bsimple(eigen_chunk[0]))
        if normalized:
            eigenvecs_latex = ', '.join(([sp.latex((mbase * eigenvec.normalized())[0]) for eigenvec in eigen_chunk[2]]))
        else:
            eigenvecs_latex = ', '.join(([sp.latex((mbase * eigenvec)[0]) for eigenvec in eigen_chunk[2]]))
        eigen_chunks.append('%s: \{{%s}\}' % (eigenval_latex, eigenvecs_latex))
    eigen_final = '\]\\\\ \n\['.join(eigen_chunks)
    eigen_final = '\[' + eigen_final + '\]'
    return eigen_final

In [99]:
def mbasis(l, astuples=False):
    '''
    Return a row matrix with symbols corresponding to the kets
    that span the angular momentum space for a given value of l.
    '''
    if astuples:
        return sp.Matrix([[(l,ml) for ml in range(-l,l+1)]])
    else:
        return sp.Matrix([[sp.Symbol('|%d,%d\\rangle' % (l,ml)) for ml in range(-l,l+1)]])

In [77]:
diagonalley = {}
l = 2
for group_num in range(1,33):
    print(group_num, end=',')
    cf = CrystalField(group_num)
    cf.matrix_rep_symb(l)
    if group_num in range(3,9):
        print("Symbolic diagonalization known to fail.")
        eigensys = []
    else:
        try:
            eigensys = cf.splitter(l)
        except:
            print("Failed in symbolic diagonalization.")
            eigensys = []
    summa = {}
    summa['simple_hams'] = cf.simplified_ham
    summa['simple_hams_latex'] = list(map(sp.latex,cf.simplified_ham))
    summa['matrices'] = cf.matrix_rep_symb(l)
    summa['splitting.latex'] = sp.latex(l_splitter(group_num, l).as_symbol_sum())
    summa['simple_matrices'] = [Bsimple(mat) for mat in cf.matrix_rep_symb(l)]
    summa['simple_matrices.latex'] = [sp.latex(smat) for smat in summa['simple_matrices']]
    summa['free_parameters'] = [mat.free_symbols for mat in summa['matrices']]
    summa['simple_free_parameters'] = [mat.free_symbols for mat in summa['simple_matrices']]
    summa['simple_free_parameters.latex'] = list(map(sp.latex,summa['simple_free_parameters']))
    summa['eigen_system'] = eigensys
    summa['eigen_latex'] = [eigen_latex(esys, l) for esys in eigensys]
    summa['group_num'] = group_num
    summa['group_symb'] = CPGs.groups[group_num].label
    diagonalley[group_num] = summa
    

1,2,3,Symbolic diagonalization known to fail.
4,Symbolic diagonalization known to fail.
5,Symbolic diagonalization known to fail.
6,Symbolic diagonalization known to fail.
7,Symbolic diagonalization known to fail.
8,Symbolic diagonalization known to fail.
9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,

In [113]:
selector = {'simple_matrices':'matrices', 'simple_free_parameters':'simple_free_parameters','free_parameters':'free_parameters','eigen_system':'eigen_system'}
crystal_pickle = {i : {selector[s]: diagonalley[i][s]  for s in selector} for i in diagonalley}
pickle.dump(crystal_pickle, open('/Users/juan/Google Drive/Zia Lab/Log/Data/crystal_splits.pkl','wb'))

In [108]:
templates = (
{'header': r'''{{\vspace{{0.5cm}}
\Large{{------------ ${group_symb}$ ({group_num}) ------------}}}}'''+'\\\\ \n',
'body0':''' \\vspace{{0.5cm}}

$V_{{CF}} = {simple_ham}$ \\\\

\[V_{{CF}}^{{l=2}}={matrix_latex}\]

\\vspace{{0.25cm}}
${splitting_latex}$ \\\\
\\vspace{{0.5cm}}
Free-parameters for d-electrons: ${simple_free_parameters}$ \\\\
\\vspace{{0.5cm}}
Eigenvalues and eigenvectors:

{eigen_latex} \n
\\vspace{{0.25cm}}
''',
'body1':''' \\vspace{{0.5cm}}
$V_{{CF}} = {simple_ham}$ \\\\
\[V_{{CF}}^{{l=2}}={matrix_latex}\]

${splitting_latex}$ \\\\
\\vspace{{0.5cm}}
Free-parameters for d-electrons: ${simple_free_parameters}$ \\\\
\\vspace{{0.25cm}}
'''})
full_latex = []
double_col_entries = [6,21]
double_col_exits = [15,32]
for group_num in diagonalley:
    summary = diagonalley[group_num]
    latex_output = [templates['header'].format(**summary)]
    if len(summary['eigen_latex']) > 0:
        for simple_ham, mat_lat, free_params, eigen_tex in zip(summary['simple_hams_latex'],
                                                   summary['simple_matrices.latex'],
                                                   summary['simple_free_parameters.latex'],
                                                   summary['eigen_latex']):
            out = templates['body0'].format(**{'simple_ham': simple_ham,
                         'simple_free_parameters': free_params, 
                         'matrix_latex': mat_lat,
                         'eigen_latex': eigen_tex,
                         'splitting_latex': summary['splitting.latex'].replace('+','\\oplus')})
            latex_output.append(out)
    else:
        for simple_ham, mat_lat, free_params, splittex in zip(summary['simple_hams_latex'],
                                                    summary['simple_matrices.latex'],
                                                    summary['simple_free_parameters.latex'],
                                                    summary['splitting.latex']):
            out = templates['body1'].format(**{'simple_ham': simple_ham,
                         'simple_free_parameters': free_params, 
                         'matrix_latex': mat_lat,
                         'splitting_latex': summary['splitting.latex'].replace('+','\\oplus')})

            latex_output.append(out)
    final_output = '\\begin{center}\n\n%s\n\\end{center}\n \\hrule \n' % (' '.join(latex_output))
    if group_num in double_col_entries:
        final_output = '\n\\begin{multicols}{2}\n' + final_output
    if group_num in double_col_exits:
        final_output = final_output + '\n\\end{multicols}\n'
    full_latex.append(final_output)
final_latex = '\n'.join(full_latex)
# pyperclip.copy(final_latex)
fname = '/Users/juan/Google Drive/Zia Lab/Theoretical Division/1delectron.tex'
open(fname,'w').write(final_latex)

48049

In [432]:
# latex_printout = {}
# for group_num in diagonalley:
#     this_chunk = diagonalley[group_num]
#     group_label = CPGs.groups[group_num].label
#     matrix_latex = []
#     for matrix in this_chunk['matrices']:
#         matrix_latex.append(sp.latex(matrix))
#     matrix_latex = '\n'.join(matrix_latex)

#### Parsing index

In [310]:
import re

In [355]:
def simple_formula_parser(job):
    '''
    Parse a chemical forumla into a dictionary of
    species strings and quantities.
    Formula cannot contain parenthesis.
    '''
    sform = job[0]
    mult = job[1]
    formula = {}
    formula['original'] = sform
    sform = re.sub(r'([1-9]+)',r' \1 ',sform)
    sform = re.sub(r'([a-z])([A-Z])','\\1 1 \\2',sform)
    sform = re.sub(r'([A-Z])([A-Z])','\\1 1 \\2',sform)
    if sform[-1] not in '1 2 3 4 5 6 7 8 9'.split(' '):
        sform = sform+' 1'
    sform = sform.strip()
    parsed = sform.split(' ')
    parsed = dict(list(zip(parsed[::2],list(map(lambda x: int(x)*mult,parsed[1::2])))))
    formula['parsed'] = parsed
    return {k:v for k,v in parsed.items() if k !=''}

In [356]:
def complex_formula_parser(sform):
    '''
    Parse a chemical forumla into a dictionary of
    species strings and quantities.
    Formula cannot contain parenthesis.
    '''
    if sform[-1] == ')':
        sform = sform + '1'
    if ')' in sform:
        multipliers = list(map(int,re.findall('\)(.)',sform)))
        groups = list(zip(re.findall(r'\((.*?)\)',sform), multipliers))
        # delete every single digit after a right parenthesis
        sform = re.sub(r'\)[0-9]',r')',sform)
        # delete every thing inside parentheses
        sform = re.sub(r'\(.+\)','',sform)
        groups = groups + [(sform,1)]
    else:
        groups = [(sform,1)]
    return list(map(simple_formula_parser,groups))
complex_formula_parser('Li2XTeO')

[{'Li': 2, 'X': 1, 'Te': 1, 'O': 1}]

In [428]:
# loader for the data in Morrison's book
# this sheet is an export of the data on 
# a Google Spreadsheet "Morrison Data"
morrison_sheets = pd.read_excel('/Users/juan/Downloads/Morrison.xlsx',None)

In [429]:
index_df = morrison_sheets['Index']
index_df['Section'] = index_df['Section'].astype(int)
# some are combinations of this and that, I shall omit these
# i am keeping the ones in which there are single substitutions of "X"
index_df = index_df[index_df['Comment1'].apply(lambda x: '*' not in str(x))]
index_df['Parsed Formula'] = index_df['Formula'].apply(complex_formula_parser)
index_df['Pretty Name'] = index_df['Pretty Name'].fillna('')

/Users/juan/opt/anaconda3/envs/qdef/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/juan/opt/anaconda3/envs/qdef/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [430]:
index = dict(zip(index_df['Section'],zip(index_df['Formula'],index_df['Parsed Formula'],index_df['Pretty Name'])))
morrison['section_to_host'] = index

In [390]:
pickle.dump(morrison,open('/Users/juan/Google Drive/Zia Lab/Codebase/qdef/data/morrison.pkl','wb'))

#### Parsing tables from Morrisons book

In [64]:
morrison_sheets = pd.read_excel('/Users/juan/Downloads/Morrison.xlsx',None)

In [65]:
morrison_keys = list(morrison_sheets)

In [66]:
def parse_comments(dframe):
    '''
    Receive a table an result a list of strings with the comments in the
    comments column.
    '''
    if 'Comments' in dframe:
        return list(filter(lambda x: x != '',list(dframe['Comments'].fillna(''))))
    else:
        print("No comments column, returning empty.")
        return []

In [73]:
def table_types(dframe):
    '''
    Receive a table and return a list with the types of thing it is.
    '''
    if 'Types' in list(dframe):
        return dframe['Types'][0].split(',')
    else:
        print("No types, returning empty.")
        return []

In [34]:
def parse_table(dframe):
    # get the type of table it is
    dframe_types = table_types(dframe)
    if ('CDATA' in dframe_types) or True:
        print("This is a table with crystallographic data.")
        # lattice params are in the first column
        crystallographic_data = list(dframe)[0]
        first_col = [s for s in dframe[list(dframe)[0]].fillna('') if s]
        lattice_params = {s.split('=')[0]:float(s.split('=')[1]) for s in first_col}
        if ('GARNET') in dframe_types:
            print("This is a table with an array of garnets.")
        else:
            

In [61]:
dframe = morrison_sheets['11.1']
crystallographic_data = list(dframe)[0]
first_col = [s for s in dframe[list(dframe)[0]].fillna('') if s]
lattice_params = {s.split('=')[0]:float(s.split('=')[1]) for s in first_col}
crystal_system = crystallographic_data.split(' ')[0].lower()

In [62]:
crystal_system

'hexagonal'

In [56]:
lattice_params

{'a': 9.206, 'c': 9.205}

In [50]:
first_col

['a=9.206', 'c=9.205']

In [35]:
parse_table(morrison_sheets['12.1'])

This is a table with crystallographic data.


#### Parsing tables with crystal field blurbs

In [501]:
# loader for the data in Morrison's book
# this sheet is an export of the data on 
# a Google Spreadsheet "Morrison Data"
morrison_sheets = pd.read_excel('/Users/juan/Downloads/Morrison.xlsx',None)

In [519]:
# these sheets parse the data of tables in Morrison
# that list values for crystal-field components
morrison_cf_components = {}
sheets_with_cf_components = [l for l in morrison_sheets if 'Crystal-field component' in list(morrison_sheets[l])[0]]
for sheetkey_with_cf_components in sheets_with_cf_components:
    sheet_with_cf_components = morrison_sheets[sheetkey_with_cf_components]
    metadata = list(sheet_with_cf_components)[0]
    host = morrison['section_to_host'][int(sheetkey_with_cf_components.split('.')[0])]
    site = metadata.split('for ')[-1]
    cf_params = dict(zip(map(sp.Symbol,sheet_with_cf_components['A_{n,m}']),
                         sheet_with_cf_components['Total']))
    cpg = re.findall(r'\((.*?)\)',site)[0]
    site_with_comments = str(site)
    site = site.split(' (')[0]
    total = {'metadata': metadata,
            'site': site,
            'site_with_comments': site_with_comments,
            'cf_params': cf_params,
            'cpg': cpg}
    morrison_cf_components[host[0]] = total

In [521]:
# these sheets provide information on experimentally
# measured values for F^2, F^4, zeta, and B_40
# the result should be something that is keyed
# by the formulas of the hosts
# and whose values are dataframes
sheets_with_exp_B40 = [l for l in morrison_sheets if 'Experimental Values' in list(morrison_sheets[l])[0]]
for sheetkey in sheets_with_exp_B40:
    print(sheetkey)
    asheet = morrison_sheets[sheetkey]
    metadata = list(asheet)[0]
    asheet = asheet[list(asheet)[1:]]
    host = morrison['section_to_host'][int(sheetkey.split('.')[0])]
    ion = asheet['Ion']

3.4
20.3


## Calculate which charge states of transition metals would give you d^n configurations

In [228]:
n=1
# returns a list of all the transition metal ions [(atomic_number, charge_state, configuration, term)]
# whose ground states includes exactly n equivalent d-electrons
tms = {}
for row  in [4,5,6]:
    tms.update(dict(map(lambda x: (x,Atom(x)),element_groups['transition metals'][row])))

In [229]:
chunks = []
for tm in tms:
    nist_datum = tms[tm].nist_data
    gstates = nist_datum[nist_datum['Level (eV)'] == 0]
    for i in range(1,11):
        if i == 1:
            seed = 'd'
        else:
            seed = 'd%d' % i
        chunk = gstates[gstates['Configuration'].apply(lambda x: (('3' + seed) in x.split('.')) or (('4' + seed) in x.split('.')) or (('5' + seed) in x.split('.')))][["Element","Charge","Configuration","Term","J"]]
        chunk["d^n"] = i
        chunk["Atomic Number"] = tm
        chunks.append(chunk)
ddframe = pd.concat(chunks)

In [230]:
# plt.close('ptable')
# font_multiplier = 2
# fig, ax = plt.subplots(num='ptable',figsize=font_multiplier*0.9*np.array([5.5,2.5]))
# margin = 0.02
# n = 1
# annotations = {atom.atomic_number :  '%.0f pm ' % atom.atomic_radius for atom in tms.values() if not(math.isnan(atom.atomic_radius))}

# for atom_num in tms:
#     atom = tms[atom_num]
#     sector = ddframe[((ddframe['d^n'] == n) | (ddframe['d^n'] == 10-n)) & (ddframe['Atomic Number'] == atom_num) & (ddframe['Charge'] < 5)]
#     charges = ','.join(list(map(lambda x: str(x)+'+',sector['Charge'])))
#     configs = ','.join(list(sector['Configuration']))
#     terms = ','.join(list(sector['Term']))
#     mdata = '\n'.join([charges, configs,terms])
#     group_loc = int(atom.group)
#     row = -atom.period
#     plt.text(group_loc+0.85, row-0.1, charges.replace(',','\n'), ha='center', va='top', fontsize=3*font_multiplier)
#     plt.text(group_loc+0.15, row-margin-0.1, terms.replace(',','\n'),ha='center',va='top',fontsize=3*font_multiplier)
#     plt.text(group_loc+0.5, row-margin-0.4, configs.replace(',','\n'),ha='center',va='top',fontsize=3*font_multiplier)
#     plt.text(group_loc+0.5, row-0.15, atom.symbol,ha='center',va='top',fontsize=7*font_multiplier, weight='bold')
#     plt.text(group_loc+0.5, row-margin-0.07, atom.atomic_number,ha='center',va='top',fontsize=1.75*font_multiplier)
#     plt.plot([group_loc+margin, group_loc+1-margin, group_loc+1-margin, group_loc+margin, group_loc+margin],
#             [ row-margin, row-margin, row-1+margin, row-1+margin, row-margin],'k-',lw=0.5)
# ax.axis('off')
# ax.set_aspect('equal')
# plt.tight_layout()
# plt.show()

In [227]:
plt.close('ptable')
font_multiplier = 2
fig, axes = plt.subplots(nrows=5, num='ptable',figsize=font_multiplier*0.9*np.array([5.5,5*1.75]))
margin = 0.02
n = 5
annotations = {atom.atomic_number :  '%.0f pm ' % atom.atomic_radius for atom in tms.values() if not(math.isnan(atom.atomic_radius))}
for n in [1,2,3,4,5]:
    ax = axes[n-1]
    for atom_num in tms:
        atom = tms[atom_num]
        sector = ddframe[((ddframe['d^n'] == n) | (ddframe['d^n'] == 10-n)) & (ddframe['Atomic Number'] == atom_num) & (ddframe['Charge'] < 5)]
        charges = ','.join(list(map(lambda x: str(x)+'+',sector['Charge'])))
        configs = ','.join(list(sector['Configuration']))
        terms = ','.join(list(sector['Term'])).replace('a ','')
        mdata = '\n'.join([charges, configs,terms])
        group_loc = int(atom.group)
        row = -atom.period
        ax.text(group_loc+0.85, row-0.1, charges.replace(',','\n'), ha='center', va='top', fontsize=3*font_multiplier)
        ax.text(group_loc+0.15, row-0.1, terms.replace(',','\n'),ha='center',va='top',fontsize=3*font_multiplier)
        ax.text(group_loc+0.5, row-margin-0.4, configs.replace(',','\n'),ha='center',va='top',fontsize=3*font_multiplier)
        ax.text(group_loc+0.5, row-0.15, atom.symbol,ha='center',va='top',fontsize=7*font_multiplier, weight='bold')
        ax.text(group_loc+0.5, row-margin-0.07, atom.atomic_number,ha='center',va='top',fontsize=1.75*font_multiplier)
        ax.plot([group_loc+margin, group_loc+1-margin, group_loc+1-margin, group_loc+margin, group_loc+margin],
                [ row-margin, row-margin, row-1+margin, row-1+margin, row-margin],'k-',lw=0.5)
        ax.axis('off')
        ax.set_aspect('equal')
        if n == 5:
            title = 'd${}^%d$' % (n)
        else:
            title = 'd${}^%d$, d${}^%d$' % (n,10-n)
        ax.set_title(title)
plt.tight_layout()

# plt.savefig('/Users/juan/Google Drive/Zia Lab/Log/Graphs/dn.pdf')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …